In [1]:
%load_ext autoreload
%autoreload 2
from mesh_import import mesh

E:\LangOn\OntoDeepLex\src\mesh\cns.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [71]:
from tqdm.autonotebook import tqdm
import numpy as np
import pandas as pd
from CwnGraph import CwnBase, CwnSense
from opencc import OpenCC
cc = OpenCC('s2t')

# Prepare a CLD character-level dataframe

In [11]:
meld_sch=pd.read_excel(mesh.get_data_dir()/"ldt-data/Tsang-2018-MELD-SCH.xlsx")
cn_nonwords = meld_sch.loc[meld_sch.lexicality==2,].word.tolist()
tw_nonwords = [cc.convert(x) for x in cn_words]
meld_sch = meld_sch.assign(trad_word=lambda df: [cc.convert(x) for x in df.word])
meld_sch = meld_sch.rename({"word": "simp_word"}, axis=1)

In [3]:
cns_dir = mesh.get_data_dir() / "cns"
cld = pd.read_csv(mesh.get_data_dir()/"CLD/chineselexicaldatabase2.1.txt")
meld_sch_avg = pd.read_csv(cns_dir / "cns_meld_sch_avg.csv", index_col=0)

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (3,4,8,9,12,13,17,18,22,23,28,29,111,112,127,128) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [172]:
cld_sub = cld.loc[:, ["Word", "C1", "C2",
        "Frequency", "C1Frequency", "C2Frequency", 
        'C1FamilySize', 'C2FamilySize', 
        'C1Friends', 'C2Friends',
        'C1FamilyFrequency', 'C2FamilyFrequency',                 
        'C1RE', 'C2RE', 
        'C1Strokes', 'C2Strokes']]
char_info_cols = ["C{}Frequency", 'C{}FamilySize',
        'C{}Friends', 'C{}FamilyFrequency', 'C{}RE', 'C{}Strokes', 'C{}Entropy']


In [173]:
cwn = CwnBase()


In [174]:
uniq_cn_characters = pd.unique(cld.loc[:, ["C1", "C2", "C3", "C4"]].melt().iloc[:,1])
uniq_cn_characters = [x for x in uniq_cn_characters if isinstance(x, str)]

In [175]:
uniq_tw_characters = [cc.convert(x) for x in uniq_cn_characters]

In [176]:
char_info = {}
from tqdm.autonotebook import tqdm
def extract_char_info(row):    
    for cidx in range(1,5):
        idx = str(cidx)
        cn_char = row["C"+idx]
        if isinstance(cn_char, str):            
            if cn_char in char_info: continue
            tw_char = cc.convert(cn_char)
            data = {'tw_char': tw_char}
            for colx in char_info_cols:                  
                cld_colname = colx.format(idx)
                std_colname = colx.replace('C{}', '')
                data[std_colname] = row.get(cld_colname, np.nan)
            data["nSense"] = len(cwn.find_all_senses(tw_char))
            char_info[cn_char] = data
cld.apply(extract_char_info, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
48639    None
48640    None
48641    None
48642    None
48643    None
Length: 48644, dtype: object

In [177]:
cld_char_info = pd.DataFrame.from_dict(char_info, orient='index').reset_index()
cld_char_info.columns.values[0] = 'cn_char'

In [178]:
cld_char_info.to_csv(cns_dir/"cld_char_info.csv")

In [179]:
cld_char_info

,cn_char,tw_char,Frequency,FamilySize,Friends,FamilyFrequency,RE,Strokes,Entropy,nSense
0,中,中,2432.6963,245.0,223.0,1408.6360,5.2031,4.0,3.9327,36
1,东,東,805.9358,101.0,101.0,978.7546,4.0809,5.0,NaN,8
2,马,馬,761.7383,171.0,171.0,471.2602,5.4438,3.0,2.0359,19
3,队,隊,286.7457,105.0,105.0,315.3669,5.1658,4.0,NaN,4
4,门,門,671.4716,195.0,194.0,723.9585,4.3092,3.0,3.6431,16
...,...,...,...,...,...,...,...,...,...,...
4890,餮,餮,5.0667,1.0,0.0,0.0148,NaN,18.0,NaN,0
4891,鳏,鰥,0.2469,1.0,0.0,0.0222,NaN,18.0,0.0000,0
4892,喑,喑,0.1235,1.0,0.0,0.0297,NaN,12.0,NaN,0
4893,枥,櫪,0.1235,1.0,0.0,0.1260,NaN,8.0,NaN,0


# Prepare a MELD-SCH dataframe with Character level info

In [180]:
meld_sch=pd.read_excel(mesh.get_data_dir()/"ldt-data/Tsang-2018-MELD-SCH.xlsx")

In [181]:
incl_cols = ['id', 'word', 'lexicality', 'N', 'RT', 'RTSD', 'zRT', 'zRTSD', 'ERR',
       'length', 'wfreq']
m2 = meld_sch.loc[meld_sch.length==2, incl_cols].reset_index()
m2["C1"] = m2.word.str.slice(0, 1)
m2["C2"] = m2.word.str.slice(1, 2)

In [182]:
m2_cld = \
    m2.merge(cld_char_info.drop('tw_char', axis=1).set_index('cn_char').add_prefix('C1'), 
             left_on="C1", right_index=True)\
      .merge(cld_char_info.drop('tw_char', axis=1).set_index('cn_char').add_prefix('C2'), 
             left_on="C2", right_index=True).drop('index', axis=1)

In [183]:
m2_cld.to_csv(cns_dir/"m2_cld.csv")

In [184]:
m2_cld

,id,word,lexicality,N,RT,RTSD,zRT,zRTSD,ERR,length,...,C1Entropy,C1nSense,C2Frequency,C2FamilySize,C2Friends,C2FamilyFrequency,C2RE,C2Strokes,C2Entropy,C2nSense
0,200001,一切,1,40,610.620250,104.615792,-0.744272,0.294925,0.000000,2,...,3.6984,10,555.2889,58.0,19.0,710.2446,5.9720,4.0,2.6764,18
10165,210166,土切,2,40,974.976250,332.479450,0.622559,1.210804,2.439024,2,...,NaN,9,555.2889,58.0,19.0,710.2446,5.9720,4.0,2.6764,18
10777,210778,手切,2,26,908.211538,357.555559,0.363349,1.037322,36.585366,2,...,NaN,6,555.2889,58.0,19.0,710.2446,5.9720,4.0,2.6764,18
11806,211807,电切,2,34,919.441176,371.619622,0.113097,0.882876,15.000000,2,...,2.8461,8,555.2889,58.0,19.0,710.2446,5.9720,4.0,2.6764,18
12624,212625,灯切,2,40,831.862750,204.859515,0.002340,0.582847,2.439024,2,...,NaN,6,555.2889,58.0,19.0,710.2446,5.9720,4.0,2.6764,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9327,209328,讥讽,1,36,784.527500,230.468194,-0.187013,0.661069,10.000000,2,...,0.9754,2,13.1802,3.0,3.0,17.4722,1.0559,6.0,NaN,0
19648,219649,转驼,2,37,1012.168108,400.822725,0.428597,1.014123,11.904762,2,...,3.9027,27,6.8840,7.0,6.0,7.5908,0.8815,8.0,1.9555,6
9980,209981,骆驼,1,39,715.283333,303.163985,-0.447445,0.825263,2.500000,2,...,NaN,1,6.8840,7.0,6.0,7.5908,0.8815,8.0,1.9555,6
14992,214993,萨茹,2,41,911.889024,319.531341,0.127748,0.813811,2.380952,2,...,NaN,0,4.4790,2.0,2.0,0.5930,NaN,10.0,NaN,0
